# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

a) Answer: 
1.  Assign each point to a cluster N at random.
2.  Calculate the mean position of each cluster using the previous
    assignments.
3.  Loop through the points - assign each point to the cluster to whose
    center it is closest.
4.  Repeat this process until the centers stop moving around.

#### b) Create the builder function for step 1.

In [ ]:
label_randomly <- function(n_points, n_clusters) {
  # assign cluster labels randomly
  labels <- ((1:n_points - 1) %% n_clusters) + 1
  sample(labels, n_points, replace = FALSE)
}

#### c) Create the builder function for step 2.

In [ ]:
get_cluster_means <- function(data, labels) {
  data %>%
    mutate(label = labels) %>%
    group_by(label) %>%
    summarise(across(everything(), mean), .groups = "drop") %>%
    arrange(label)
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [ ]:
assign_cluster <- function(data, means) {
  data_mat <- as.matrix(data)
  means_mat <- as.matrix(means %>% select(-label))
  
  dist_mat <- outer(
    1:nrow(data_mat), 1:nrow(means_mat),
    Vectorize(function(i, j) sum((data_mat[i, ] - means_mat[j, ])^2))
  )
      
  labels <- means$label[apply(dist_mat, 1, which.min)]
  labels
}

#### e) Create the builder function for step 4.

In [ ]:
kmeans_done <- function(old_means, new_means, eps = 1e-6) {
  om <- as.matrix(old_means %>% select(-label))
  nm <- as.matrix(new_means %>% select(-label))
  
  m <- mean(sqrt(rowSums((om - nm)^2)))
  m < eps
}

#### f) Combine them all into your own `kmeans` function.

In [ ]:
mykmeans <- function(data, n_clusters, eps = 1e-6) {
  labels <- label_randomly(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE
  
  while (!done) {
    labels <- assign_cluster(data, old_means)
    new_means <- get_cluster_means(data, labels)
    
    if (kmeans_done(old_means, new_means, eps)) {
      done <- TRUE
    } else {
      old_means <- new_means
    }
  }
  
  list(labels = labels, means = new_means)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set. 

In [ ]:
voltages_df <- read_csv("~/Desktop/BIOS512/Homeworks/Homework 8/voltages_df.csv")

#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`. 

In [ ]:
results <- mykmeans(voltages_df, n_clusters = 3)
print(results$labels)
print(results$means)

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`. 
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [ ]:
voltages_matrix <- as.matrix(voltages_df)
results2 <- kmeans(voltages_matrix, centers = 3)

print(results2$centers)   
print(results2$cluster)

#### d) Are your labels/clusters the same? If not, why? Are your means the same?

Answer: No, the labels/clusters are not the same because cluster numbers are arbitrary and randomly assigned labels, not meaningful identifiers. Even if both algorithms work correctly, K-means is not guaranteed to assign the same label (e.g., cluster 1 vs. cluster 2) to the same group. Since my mykmeans function uses its own initialization logic, the order of cluster centers naturally differs from R’s kmeans output. The mean values are the same, but they are associated with different cluster numbers due to the random nature of label assignment.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

Answer: When using a for loop to assign clusters in kmeans, you loop over each data point (i) and, for each one, loop over all cluster centers (j) to calculate the distance between the point and each cluster mean. The point is then assigned to the cluster with the smallest distance. This process involves a double loop, one over all data points (N) and another over all clusters (K), which makes it computationally slow, especially for large datasets.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

Answer: When vectorizing the code to assign clusters in kmeans, instead of looping through each point and cluster, you create two large replicated matrices so all point–cluster pairs can be compared simultaneously. The first matrix, Data′, replicates each data point K times (once for each cluster), and the second matrix, Means′, replicates each cluster mean N times (once for each data point). These matrices now have the same dimensions (N × K × D), allowing you to subtract them directly and compute all distances in a single vectorized operation. This approach eliminates explicit loops and greatly improves computational efficiency.

#### c) State which (for loops or vectorizing) is more efficient and why.

Answer: Vectorizing is more efficient because it eliminates the need for slow double loops over N and K. Instead of iterating through each data point and cluster, vectorization uses matrix operations. This allows the replicated matrices to be generated and all distances computed simultaneously, making the process much faster and more efficient than using explicit loops.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

Answer: Kmeans fails when the data are not well separated, not spherical, or when clusters differ greatly in size or density. Kmeans assumes that the data are vector-based and that clusters are spherical (uniformly shaped Gaussians) of roughly equal size and variance. It also assumes that each point belongs to exactly one cluster. These assumptions cause Kmeans to perform poorly when clusters overlap, have irregular shapes, or vary in size and density.

## Question 5
#### What assumption do Guassian mixture models make?

Answer: A Gaussian mixture model assumes that the data is drawn from a mixture of N Gaussian (normal) distributions, each with its own mean and covariance. These distributions represent different clusters, and their parameters  are estimated from the data. This assumption allows GMMs to model clusters of varying sizes, shapes, and orientations more flexibly than kmeans.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

Answer: Spectral clustering makes the assumption that two points are more likely to be in the same cluster if they are close to one another. The mere existence of a metric is a much weaker condition than the existence of a vector space, and thus we can work with substantially more types of data and complex data structures. 

## Question 7
#### Define the gap statistic method. What do we use it for?

Answer: The gap statistic method is a standard method of calculating the cluster number. It compares the clustering for each value of K to a cluster of data "randomized" into the same domain as the original data. Then it compute the dispersion of the two clusterings and look at the difference. The “gap” is the difference between the observed and expected dispersion. The optimal number of clusters is typically chosen at the point where this gap is largest (the “knee”), indicating that the clustering structure better than what would be expected by chance.